## Remember to open terminal and run: conda activate ilab-tensorflow to import shap

In [1]:
import shap

In [2]:
import sys
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion')
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/monte-carlo/monte-carlo')
#sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion')

In [3]:
from multi_path_fusion.src.training.train_gt import train

from multi_path_fusion.src.utils.mlflow_helpers import setup_mlflow, log_params
from multi_path_fusion.src.utils.data_generator_helpers import load_data_generator
from multi_path_fusion.src.utils.model_helpers import get_model_factory

2024-01-15 10:10:52.148198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 10:10:52.328308: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
from tensorflow import keras
import json
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import rasterio
import pickle
from deepdiff import DeepDiff
from pprint import pprint
import tensorflow as tf

In [20]:
# def configure_virtual_cpus(ncpu):
#   phy_devices = tf.config.list_physical_devices('CPU')
#   tf.config.set_logical_device_configuration(phy_devices[0], [
#         tf.config.LogicalDeviceConfiguration(),
#     ] * ncpu)

# configure_virtual_cpus(8)
# DEVICES = [f'CPU:{i}' for i in range(8)]

# tf.config.list_logical_devices('CPU')

In [26]:
with open('/panfs/ccds02/home/gtamkin/dev/AGB/monte-carlo/monte-carlo/tests/exp_7bands.json', 'r') as f:
    config = json.load(f)
    models_config = config["models"]
    data_generator_config = config["data_generator"]
    mlflow_config = config["mlflow"]

In [27]:
# load train, validate, and test data
train_generator = load_data_generator(data_generator_config, 'train')
validate_generator = load_data_generator(data_generator_config, 'validate')
test_generator = load_data_generator(data_generator_config, 'test')

reading in band: 2
reading in band: 16
reading in band: 2
reading in band: 16
reading in band: 2
reading in band: 16


In [28]:
print(mlflow_config)
print(data_generator_config)
print(models_config)


{'TRACKING_URI': '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands_load_model', 'EXPERIMENT_NAME': 'Exp_7bands_stats[18,14,9,3,20,8,10]'}
{'branch_inputs': [{'branch_files': [{'mlbs_year_filepath': 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices.tif', 'bands1': [18, 14, 9, 3, 20, 8, 10], 'bands2': [17, 4, 1, 15, 6, 7, 11], 'bands3': [16, 13, 12, 2, 5, 19, 21], 'bands': [2, 16], 'bands7': [11, 16, 9, 10, 8], 'bands6': [12, 2, 4, 21, 13, 14, 17, 19, 1, 7, 18, 16, 20, 6, 15, 9, 5, 3, 8, 10, 11], 'bands5': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]}]}], 'data_path': '/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/', 'dataset_width': 1000, 'dataset_height': 1000, 'patch_width': 1, 'patch_height': 1, 'batch_size': 50, 'full_hyperspectral': False, 'num_bands': 5, 'truth_file_a': 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_FSDGtif_CHM_warp.tif', 'truth_file_b': 'MLBS_2021_541567.6_4136443.0_

In [29]:
model_config = models_config[0]

mlflow_config = setup_mlflow(mlflow_config)
print(model_config, mlflow_config)

{'model_name': 'MLP_7bands_2_epochs', 'model_type': 'Sequential', 'model_description': 'Testing the Keras functionality after this whole architecture redesign', 'layers': [{'type': 'Dense', 'units': 5, 'return_sequences': True, 'activation': 'gelu', 'kernel_initializer': 'he_normal', 'use_bias': True}, {'type': 'Dense', 'units': 5, 'activation': 'softmax'}], 'compile_options': {'optimizer': 'adam', 'loss': 'sparse_categorical_crossentropy'}, 'training_options': {'epochs': 1}, 'callbacks': {'EarlyStopping': {'monitor': 'val_loss', 'patience': 20}}} {'TRACKING_URI': '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands_load_model', 'EXPERIMENT_NAME': 'Exp_7bands_stats[18,14,9,3,20,8,10]', 'EXPERIMENT_ID': '533710230248440706'}


In [30]:
    import time  
    
    model_type = model_config.get("model_type", "Sequential")  
    model_factory = get_model_factory(model_type)  
    model = model_factory.create_model(model_config, data_generator_config)
    model = model_factory.compile_model(model, model_config)

    # TODO: potentially add an option for being able to load in a previous model 
    # (such as using an encoder previously and training on the latent space instead of the original data)

    # train_generator = load_data_generator(data_generator_config, 'train')
    # validate_generator = load_data_generator(data_generator_config, 'validate')
    # test_generator = load_data_generator(data_generator_config, 'test')

    # probably not needed - will delete soon
    # run_name = get_unique_run_name(mlflow_config, model_config.get('model_name', None))
    with mlflow.start_run(experiment_id=mlflow_config['EXPERIMENT_ID'], run_name=model_config.get("model_name", None), description=model_config.get("model_description", ""), nested=False) as run:
        model_factory.autolog()
        log_params(model_config)
        RUN_ID = mlflow.active_run().info.run_id
        print(f"STARTING RUN: {RUN_ID}")
        
        # TODO: figure out best spot to put this function in, or if there needs to be multiple
        # "extras" depending on where in the training flow you want it to occur
        # put any miscellaneous project-specific code here
        model_factory.extras(model, train_generator, validate_generator, test_generator)

        t0 = time.time()

        # TODO: figure out a way to make this if statement more generic 
        # problem is keras model summary best done before training, xgboost summary only able to be done after
        if model_type == "Sequential" or "Keras":
            model_factory.summary(model)
        history = model_factory.train_model(model, train_generator, validate_generator, model_config)
 #       history2 = model_factory.train_model(model, train_generator, validate_generator, model_config)
        print(f'history: {history.history}')
 #       print(f'history2: {history2.history}')

        t1 = time.time()
        total_time = t1 - t0

        mlflow.log_param("Training time", total_time)

        print()
        print("Total time for model.fit() processing is: " + str(total_time) + " seconds.")

        if model_type == "XGBoost":
            model_factory.summary(model)

        predictions = model_factory.predict(model, test_generator)
        test_results = model_factory.evaluate(model, test_generator)
        
        # debugging
        print(f'predictions: {predictions}')
        print(f'test results: {test_results}')

        model_factory.log_metrics(test_results)
        model_factory.plot_metrics(model, test_generator, history)

        
        # debugging
        print("Finished successfully")


STARTING RUN: 51db891307ea46f98b9537b458177419
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 30        
                                                                 
 dense_3 (Dense)             (None, 5)                 30        
                                                                 
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


2024/01/15 11:18:55 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'multi_path_fusion.src.data_generators.mpf_h_data_generator.MPF_H_DataGenerator'>. Dataset logging skipped.
2024/01/15 11:18:55 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'multi_path_fusion.src.data_generators.mpf_h_data_generator.MPF_H_DataGenerator'>. Dataset logging skipped.


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/dense_2/BiasAdd' defined at (most recent call last):
    File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/panfs/ccds02/app/modules/j

In [10]:
branch_num = file_num = 0
full_tile = data_generator_config['branch_inputs'][branch_num]["branch_files"][file_num]["mlbs_year_filepath"]
full_tile_prefix = os.path.splitext(full_tile)  # returns ('/home/user/somefile', '.txt')
tile_prefix = str(full_tile_prefix[0]) 
print(tile_prefix)

MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices


In [11]:
archive_id = mlflow_config['TRACKING_URI'] +  "/" + tile_prefix + "/" + model_config.get("model_name") + '::' + mlflow_config['EXPERIMENT_ID'] + '/'
if not os.path.exists(archive_id):
    os.makedirs(archive_id)
print(archive_id)

/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices/MLP_7bands_2_epochs::138094747525594611/


In [40]:
import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)
print("Your Computer Name is:" + hostname)

archive_model_id = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model'
# another_strategy = tf.distribute.MirroredStrategy()
# with another_strategy.scope():
#     load_options = tf.saved_model.LoadOptions(experimental_io_device=hostname)
#     loaded = tf.keras.models.load_model(archive_model_id, options=load_options)

# load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
# loaded = tf.keras.models.load_model(archive_model_id, options=load_options)

loaded = tf.keras.models.load_model(archive_model_id)


Your Computer Name is:gpu004


FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

# Save/restore model() with tensorflow.keras.models() and pickle().  Gets errors with both approaches (see below).

In [33]:
archive_model_id = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model'
model.save(archive_model_id)
model_rk = tf.keras.models.load_model(archive_model_id)
print(archive_model_id, model, model_rk)

INFO:tensorflow:Assets written to: /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model/assets


INFO:tensorflow:Assets written to: /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model/assets


FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [12]:
archive_model_id = archive_id + model_config.get("model_name") + ".keras_k"
model.save(archive_model_id)
model_rk = tf.keras.models.load_model(archive_model_id)
print(archive_model_id, model, model_rk)

INFO:tensorflow:Assets written to: /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices/MLP_7bands_2_epochs::138094747525594611/MLP_7bands_2_epochs.keras_k/assets


INFO:tensorflow:Assets written to: /explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices/MLP_7bands_2_epochs::138094747525594611/MLP_7bands_2_epochs.keras_k/assets


/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices/MLP_7bands_2_epochs::138094747525594611/MLP_7bands_2_epochs.keras_k <keras.engine.sequential.Sequential object at 0x1460c88c9af0> <keras.engine.sequential.Sequential object at 0x145ff5de8f10>


In [17]:
# print(model)
# differences = DeepDiff(model, model_rk)
# pprint(model, differences)

In [14]:
archive_model_id = archive_id + model_config.get("model_name") + ".keras_p"
pickle.dump(model, open(archive_model_id, "wb"))
model_rp = pickle.load(open(archive_model_id, "rb"))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........2
.........3
.........4
.........5
.........6
.........7
.........8
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2024-01-15 10:14:58           64
config.json                                    2024-01-15 10:14:58         1372
variables.h5                                   2024-01-15 10:14:58        21392
Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2024-01-15 10:14:58           64
config.json   

In [15]:
print('Model I/O: ')
print('\nmodel -> trained model: ')
print(model.weights)

print('\nmodel_r -> trained model: ')
print(model_rk.weights)

Model I/O: 

model -> trained model: 
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[-0.04904908, -0.73021334,  0.08650863, -0.49822754, -0.69361174],
       [ 0.7660165 , -0.17820251, -0.26483762,  0.42923212, -0.3845378 ],
       [-0.25372738, -0.51191515,  0.21147764, -0.6425986 , -0.56746954],
       [ 0.26987362, -0.57026625,  0.4126774 , -0.27614143, -0.7255419 ],
       [ 0.25262725, -0.38714802,  0.06436145, -0.22253472, -0.02028376]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[-0.27380675, -0.64187217, -0.30194208, -0.5833962 , -0.49994192],
       [-0.26948434,  0.19278693,  0.6494565 ,  0.6940801 , -0.2629711 ],
       [ 0.7647644 , -0.21074575, -0.11840165, -0.63423127,  0.48843348],
       [ 0.04930228,  0.7143147 ,  0.50581765, -0.5671184 ,  0.56588864],
       [-0.63720566,  0.0790351 , 

In [16]:
# differences = DeepDiff(model.weights, model_rk.weights)
# pprint(model, differences)

TypeError: Cannot iterate over a scalar tensor.

## Retrain each restored model and compare history.  The histories were saved cleanly.
### Histories from restored models seem to reflect that they were saved sufficiently despite tracebacks about certain attributes.  But use the Keras() to save instead of pickle to be safe. 

In [ ]:
        history_rk = model_factory.train_model(model_rk, train_generator, validate_generator, model_config)
        history_rp = model_factory.train_model(model_rp, train_generator, validate_generator, model_config)
        print(f'history_rk: {history_rk.history}')
        print(f'history_rp: {history_rp.history}')


In [ ]:
differences = DeepDiff(history.history, history2.history)
pprint(differences)

In [ ]:
differences = DeepDiff(history_rk.history, history_rp.history)
pprint(differences)

## Save/restore each trained model history and compare results using pickle().  The history was saved cleanly.

In [ ]:
archive_history_id = archive_id + model_config.get("model_name") + "_history.fit"
pickle.dump(history, open(archive_history_id, "wb"))
history_r = pickle.load(open(archive_history_id, "rb"))

In [ ]:
differences = DeepDiff(history.history, history_r.history)
pprint(differences)

## Save/restore test_results() from each evaluation using pickle(). The test_results() were saved cleanly.

In [ ]:
archive_evaluate_id = archive_id + model_config.get("model_name") + "_evaluate.results"
pickle.dump(test_results, open(archive_evaluate_id, "wb"))
test_results_r = pickle.load(open(archive_evaluate_id, "rb"))

In [ ]:
differences = DeepDiff(test_results, test_results_r)
pprint(differences)

In [ ]:
print('History I/O: ')
print('\nhistory -> trained model history: ')
print('epoch', history.epoch)
print('history', history.history)
print('model.weights', history.model.weights)
print('params', history.params)

# print('\nhistory_r -> trained model history: ')
# print('epoch', history_r.epoch)
# print('history', history_r.history)
# print('model.weights', history_r.model.weights)
# print('params', history_r.params)

In [ ]:
print('Evaluate I/O: ')
print('\nevaluate -> test_results: ')
print('test loss', test_results['test loss'])
print('test accuracy', test_results['test accuracy'])

# print('\nevaluate_r -> test_results: ')
# print('test loss', test_results_r['test loss'])
# print('test accuracy', test_results_r['test accuracy'])


## Save/restore test_generator() data using pickle. The test_generator() was saved cleanly.

In [ ]:
archive_test_generator_id = archive_id + model_config.get("model_name") + "_test_generator.data"
pickle.dump(test_generator, open(archive_test_generator_id, "wb"))
test_generator_r = pickle.load(open(archive_test_generator_id, "rb"))

In [ ]:
differences = DeepDiff(test_generator, test_generator_r)
pprint(differences)

In [ ]:
print('Test Generator I/O: ')
print('\ntest_generator -> data: ')
print('batch_size', test_generator.batch_size)
print('binned', test_generator.binned)
print('len(binned)', len(test_generator.binned))
print('branch_inputs', test_generator.branch_inputs)
print('file_x_stack.shape', test_generator.file_x_stack.shape)
print('file_x_stack', test_generator.file_x_stack)
print('file_x_stack.min()', test_generator.file_x_stack.min())
print('file_x_stack.max()', test_generator.file_x_stack.max())
print('file_x_stack.mean()', test_generator.file_x_stack.mean())
print('file_x_stack[6].mean()', test_generator.file_x_stack[6].mean())
print('truthset_a', test_generator.truthset_a)

print('\ntest_generator_r -> data: ')
# print('batch_size', test_generator_r.batch_size)
# print('binned', test_generator_r.binned)
# print('len(binned)', len(test_generator_r.binned))
# print('branch_inputs', test_generator_r.branch_inputs)
# print('file_x_stack.shape', test_generator_r.file_x_stack.shape)
# print('file_x_stack', test_generator_r.file_x_stack)
# print('file_x_stack.min()', test_generator_r.file_x_stack.min())
# print('file_x_stack.max()', test_generator_r.file_x_stack.max())
# print('file_x_stack.mean()', test_generator_r.file_x_stack.mean())
# print('file_x_stack[6].mean()', test_generator_r.file_x_stack[6].mean())
print('truthset_a', test_generator_r.truthset_a)

## 12/19/2023 - ONTO SHAP VALUES NOW THAT WE'VE CREATED, TRAINED (fit), EVALUATED AND STORED THE MODEL, MODEL HISTORY and TEST-GENERATOR DATA.  
### NOTE: PREDICTIONS NOT IMPORTANT UNTIL PERMUATION IMPORTANCE IS IDENTIFIED (ala MONTE CARLO)

In [ ]:
import shap
keywordAll = ['ARI', 'CAI', 'CRI550', 'CRI700', 'EVI', 'EVI2', 'fPAR', 'LAI', 'MCTI', 'MSI',
                'NDII', 'NDLI', 'NDNI', 'NDVI', 'NDWI', 'NIRv', 'PRIn', 'PRIw', 'SAVI', 'WBI', 'Albedo']
keywordAll

In [ ]:
def print_feature_importances_shap_values(shap_values, features):
    
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''

    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
        
    # Calculates the normalized version
    importances_norm = softmax(importances)

    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}

    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}

    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

In [ ]:
def evaluate_regression(y, y_pred):
    
    '''
    Prints the most common evaluation metrics for regression
    '''
    
    mae = MAE(y, y_pred)
    mse = MSE(y, y_pred)
    rmse = mse ** (1/2)
    r2 = R2(y, y_pred)
    
    print('Regression result')
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2: {r2:.2f}")

## Convert test_generator.file_x_stack to Pandas dataframe and transpose for SHAP API.

In [ ]:
import pandas as pd
df = pd.DataFrame(data=test_generator_r.file_x_stack)
dft = df.transpose()
X = X_test = dft
print(df.shape, X.shape)

In [ ]:
#     def _get_shap_values(self, model, test_generator, X):



#         import shap

#         explainer = shap.KernelExplainer(model.predict, X.iloc[:50, :])

#         self.mpfConfig.shap_values0to50 = self.mpfConfig.explainer.shap_values(X.iloc[0:50, :], nsamples=500)
#         # self.mpfConfig.shap_values51to100 = self.mpfConfig.explainer.shap_values(X.iloc[50:99, :], nsamples=500)
#         # self.mpfConfig.shap_value101to150 = self.mpfConfig.explainer.shap_values(X.iloc[100:149, :], nsamples=500)
#         # self.mpfConfig.shap_values50 = self.mpfConfig.explainer.shap_values(X.iloc[280:330, :], nsamples=500)

#         shap.summary_plot(self.mpfConfig.shap_values0to50[0], X.iloc[0:50, :], plot_type="bar", feature_names=keyword)
#         shap.summary_plot(shap_values0to50[0], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)
#         print(len(self.mpfConfig.shap_values))
#         print(self.mpfConfig.shap_values)
#         return self.mpfConfig.explainer, self.mpfConfig.shap_values


# Use Explainer to explain the output 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from scipy.special import softmax

In [ ]:
# Evaluates the model
y_pred = model.predict(X_test)
#evaluate_regression(X_test, y_pred)

In [ ]:
print(y_pred, y_pred.shape)

In [ ]:
# Fits the explainer
explainer = shap.Explainer(model.predict, X_test)

In [ ]:
# Calculates the SHAP values - It takes some time
shap_values = explainer(X_test.iloc[:50, :])

# Use KernelExplainer to explain the output of any function. https://shap-lrjball.readthedocs.io/en/latest/generated/shap.KernelExplainer.html

## Kernel SHAP is a method that uses a special weighted linear regression to compute the importance of each feature. The computed importance values are Shapley values from game theory and also coefficents from a local linear regression.

## Parameters:
### model [function or iml.Model]:
### User supplied function that takes a matrix of samples (# samples x # features) and computes a the output of the model for those samples. The output can be a vector (# samples) or a matrix (# samples x # model outputs).

### data [datanumpy.array or pandas.DataFrame or shap.common.DenseData or any scipy.sparse matrix]:
### The background dataset to use for integrating out features. To determine the impact of a feature, that feature is set to “missing” and the change in the model output is observed. Since most models aren’t designed to handle arbitrary missing data at test time, we simulate “missing” by replacing the feature with the values it takes in the background dataset. So if the background dataset is a simple sample of all zeros, then we would approximate a feature being missing by setting it to zero. For small problems this background dataset can be the whole training set, but for larger problems consider using a single reference value or using the kmeans function to summarize the dataset. Note: for sparse case we accept any sparse matrix but convert to lil format for performance.

In [ ]:
# See above.  In this call, we use the first 50 rows of the test-generator dataset as the background dataset.  Should be configurable.  QUESTION: Should this be train_generator instead (perhaps with K-means)
explainer = shap.KernelExplainer(model.predict, X.iloc[:50, :])
print(explainer)

## Save/restore KernelExplainer data using pickle().  Explainer saved cleanly.

In [ ]:
archive_explainer_id = archive_id + model_config.get("model_name") + "_kernel.explainer"
pickle.dump(explainer, open(archive_explainer_id, "wb"))
explainer_r = pickle.load(open(archive_explainer_id, "rb"))

In [ ]:
differences = DeepDiff(explainer, explainer_r)
pprint(differences)

In [ ]:
print('KernelExplainer I/O: ')
print('\n Explainer -> features: ')
print('data_feature_names', explainer.data_feature_names)
print('len(expected_value)', len(explainer.expected_value))
print('expected_value', explainer.expected_value)

print('\n Explainer_r -> features: ')
print('data_feature_names', explainer_r.data_feature_names)
print('len(expected_value)', len(explainer_r.expected_value))
print('expected_value', explainer_r.expected_value)


In [ ]:
archive_explainer_id

# Estimate the SHAP values for a set of samples:  shap_values(X, **kwargs)
## https://shap.readthedocs.io/en/latest/generated/shap.KernelExplainer.html#shap.KernelExplainer.shap_values

## Parameters:
## Xnumpy.array or pandas.DataFrame or any scipy.sparse matrix
### A matrix of samples (# samples x # features) on which to explain the model’s output.
## nsamples“auto” or int
### Number of times to re-evaluate the model when explaining each prediction. More samples lead to lower variance estimates of the SHAP values. The “auto” setting uses nsamples = 2 * X.shape[1] + 2048.
## gc_collectbool
### Run garbage collection after each explanation round. Sometime needed for memory intensive explanations (default False).
## Returns:  array or list
### For models with a single output this returns a matrix of SHAP values (# samples x # features). Each row sums to the difference between the model output for that sample and the expected value of the model output (which is stored as ## expected_value attribute of the explainer). For models with vector outputs this returns a list of such matrices, one for each output.  

# *NOTE:  In our nomenclature, features = bands = indices*

In [ ]:
# Get the shap values for the first 50 rows in the test-generator dataset - based on 500 samples each (should be configurable)
shap_values0to50 = explainer.shap_values(X.iloc[0:50, :], nsamples=500)


In [ ]:
# sshap_values0to50.shape() = (50, 7). - first 50 rows by 7 band columns
# there are 5 rows of shape values because binning results in 5 classifications
#print(len(shap_values0to50), shap_values0to50[0].shape , shap_values0to50[4][0].shape)
lastShapRow = shap_values0to50[0][0]
print(lastShapRow.shape, lastShapRow[0].max(), lastShapRow)
sum(explainer_r.expected_value[0] - lastShapRow)

In [ ]:
bandLen = 20

bandOccurenceArr = np.zeros(bandLen)
print(bandOccurenceArr)
bandMaxArr = np.zeros(bandLen)
bandMinArr = np.zeros(bandLen)
bandMeanArr = np.zeros(bandLen)


In [ ]:
explainer.explain_row(0)

In [ ]:
#shap_values0to50 = explainer.shap_values(X.iloc[0:50, :], nsamples=500)
shap_values_explanation = explainer(X.iloc[0:50, :])

In [ ]:
#print(shap_values_explanation[0], shap_values_explanation[0].values[0][0],  shap_values_explanation[0].data[0])
print(shap_values_explanation[0])

In [ ]:
base_delta = (shap_values_explanation.base_values[0] - shap_values_explanation.values[0])
pprint(base_delta)
col_totals = [ sum(x) for x in zip(*base_delta) ]
pprint(col_totals)

In [ ]:
print('expected_value', explainer_r.expected_value)
print(' - expected_value', explainer_r.expected_value)


In [ ]:
shap_values_explanation

In [ ]:
print(X.iloc[0:, :])

In [ ]:
archive_shap_values_explanation_id = archive_id + model_config.get("model_name") + "_shap.explanation"
pickle.dump(shap_values_explanation, open(archive_shap_values_explanation_id, "wb"))
archive_shap_values_explanation_id_r = pickle.load(open(archive_shap_values_explanation_id, "rb"))

In [ ]:
differences = DeepDiff(shap_values_explanation, shap_values_explanation_id_r)
pprint(differences)

## Repeat logic using X_train instead of parts of X_test

In [ ]:
import pandas as pd 
df = pd.DataFrame(data=train_generator.file_x_stack)
dft = df.transpose()
X_train = dft
print(df.shape, X_train.shape)

In [ ]:
# See above.  In this call, we use the first 50 rows of the test-generator dataset as the background dataset.  Should be configurable.
explainer_X_train = shap.KernelExplainer(model.predict, X_train)
#explainer_X_train = shap.KernelExplainer(model.predict, X_train.iloc[:50, :])
print(explainer_X_train)

In [ ]:
shap_values0to50_X_train = explainer.shap_values(X.iloc[0:50, :], nsamples=500)


In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model  
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
def print_accuracy(f):
    print(
        "Root mean squared test error = {}".format(
            np.sqrt(np.mean((f(X_test) - y_test) ** 2))
        )
    )
    time.sleep(0.5)  # to let the print get out before any progress bars


shap.initjs()

# ===== END: GENERAL Model creation, training, prediction, evaluation from MPF above

# ==== MAIN THREAD FROM 11/25: shap.KernelExplainer
## https://jh-ml.nccs.nasa.gov/jupyterhub-prism/user/gtamkin/lab/tree/_AGB-dev/shap/notebooks/tabular_examples/neural_networks/Census%20income%20classification%20with%20Keras.ipynb
## https://stackoverflow.com/questions/45361559/feature-importance-chart-in-neural-network-using-keras-in-python

In [ ]:
import pandas as pd
df = pd.DataFrame(data=test_generator.file_x_stack)
df

In [ ]:
dft = df.transpose()
dft.iloc[:, :]

In [ ]:
X = dft
print(X.shape)
print(X.shape[0])
print(X.shape[1])

In [ ]:
print(X.shape[1])
count = 0
for i in range(X.shape[1]):
    count = count + 1

print(count)
#print(X[:,count])

In [ ]:
#print([X[:, i] for i in range(X.shape[1])])
print(X.shape)
print(X.iloc[:, :])
print(X.iloc[:5, :])
print(X.iloc[0, 0])
print(X.iloc[3, 6])
print(X.iloc[:3, :])

In [ ]:
def my_predict2(X):
    return model.predict([X[:, i] for i in range(X.shape[1])]).flatten()

In [ ]:
import shap

# load your data here, e.g. X and y
# create and fit your model here

# load JS visualization code to notebook
shap.initjs()
print(model.summary())

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.KernelExplainer(model.predict, X.iloc[:50, :])
#explainer = shap.KernelExplainer(my_predict2, X.iloc[:2, :])

In [ ]:
#print(X.iloc[:50, :])
shap_values = explainer.shap_values(X.iloc[299, :], nsamples=500)
shap_values2 = explainer.shap_values(X.iloc[299, :], nsamples=500)
print('shap_values: '+ str(len(shap_values)))
print(shap_values)
print('shap_values2: '+ str(len(shap_values2)))
print(shap_values2)

In [ ]:
X.iloc[299, :]

In [ ]:
shap_values[0].tofile('./mlruns/shap1.shap')

In [ ]:
shap_values0 = np.fromfile('./mlruns/shap1.shap')

In [ ]:
print(train_generator.file_x_stack.shape, train_generator.file_x_stack[0].shape)

In [ ]:
print(shap_values[0], sum(shap_values[0]))
print(shap_values0, sum(shap_values0))

In [ ]:
print(explainer.fx, sum(explainer.fx))
print(explainer.expected_value, sum(explainer.expected_value))
diff = explainer.fx - explainer.expected_value
print(diff, sum(diff))

In [ ]:
padding = 3
bandList = data_generator_config['branch_inputs'][0]['branch_files'][0]['bands']
bandList

In [ ]:
bandRootDir = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/12122023-7b/BANDS'
hyperspectralIndicesFile = data_generator_config['branch_inputs'][0]['branch_files'][0]['mlbs_year_filepath'] 
print(hyperspectralIndicesFile)
hpath, hname = hyperspectralIndicesFile.split('/',1)
hprefix, hsuffix = hname.split('.',1)
print(bandRootDir, hpath, hprefix)
newPath = os.path.join(bandRootDir, hpath)
print(newPath)
if (not os.path.exists(newPath)): os.makedirs(newPath)

# Write out bands
for i in range(len(bandList)):
    newPathFile = os.path.join(newPath, hprefix+'_band'+str(bandList[i]).zfill(padding)+'.band')
    print(i, str(bandList[i]).zfill(padding), newPathFile, train_generator.file_x_stack[i].dtype, train_generator.file_x_stack[i].shape, train_generator.file_x_stack[i].min())
    train_generator.file_x_stack[i].tofile(newPathFile)

In [ ]:
# Read in bands
for i in range(len(bandList)):
    existingBandFile = os.path.join(newPath, hprefix+'_band'+str(bandList[i]).zfill(padding)+'.band')
    bandValues = np.fromfile(existingBandFile)
    print(existingBandFile, bandValues.dtype, bandValues.shape, bandValues.min())

In [ ]:
train_generator.file_x_stack[0].max()

In [ ]:
train_generator.file_x_stack[1].max()

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()

In [ ]:
band018 = train_generator.file_x_stack[0]
band014 = train_generator.file_x_stack[1]
band009 = train_generator.file_x_stack[2]
band003 = train_generator.file_x_stack[3]
band020 = train_generator.file_x_stack[4]
band008 = train_generator.file_x_stack[5]
band010 = train_generator.file_x_stack[6]
print(band018, band018.dtype, band018.shape)

In [ ]:
np.savez(outfile, band018=band018, band014=band014, band009=band009, band003=band003, band020=band020, band008=band008, band010=band010)

In [ ]:
_ = outfile.seek(0)
print(outfile)

In [ ]:
npzfile = np.load(outfile)

In [ ]:
npzfile.files

In [ ]:
print(band018, band018.dtype, band018.shape, band018.max())
print(npzfile['band018'], npzfile['band018'].dtype, npzfile['band018'].shape, npzfile['band018'].max())

In [ ]:
from tempfile import TemporaryFile
band018file = TemporaryFile()

In [ ]:
np.save(band018file, train_generator.file_x_stack[0])

In [ ]:
_ = band018file.seek(0)
b18 = np.load(band018file)
print(b18, b18.dtype, b18.shape, b18.max())

In [ ]:
explainer.expected_value

In [ ]:
explainer.expected_value[0]

In [ ]:
#1    Index Name=ARI
#2    Index Name=CAI
#3    Index Name=CRI550
#4    Index Name=CRI700
#5    Index Name=EVI
#6    Index Name=EVI2
#7    Index Name=fPAR
#8    Index Name=LAI
#9    Index Name=MCTI
#10    Index Name=MSI
#11   Index Name=NDII
#12    Index Name=NDLI
#13    Index Name=NDNI
#14    Index Name=NDVI
#15    Index Name=NDWI
#16    Index Name=NIRv
#17    Index Name=PRIn
#18    Index Name=PRIw
#19    Index Name=SAVI
#20    Index Name=WBI
#21    Index Name=Albedo

#        "bands": [18, 14, 9, 3, 20, 8, 10]}]
    
keyword = list()
keyword.append('PRIw')
keyword.append('NDVI')
keyword.append('MCTI')
keyword.append('CRI550')
keyword.append('WBI')
keyword.append('LAI')
keyword.append('MSI')
print(keyword)

In [ ]:
x = 0
while x < 5:
    diff = explainer.expected_value[x] - shap_values[x]
    print("explainer.expected_value[x] where x = " + str(x))
    print(explainer.expected_value[x], shap_values[x], diff, 
      diff.min(), diff.max())
    x = x + 1
print(x)
print(explainer.expected_value, sum(explainer.expected_value))
print(explainer.fx, sum(explainer.fx))

In [ ]:
print(X.shape)

In [ ]:
# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
#X_test_kmeans_summary = shap.kmeans(X, 1000)
#print(X_test_kmeans_summary[0])

In [ ]:
#print(X_test_kmeans_summary[0])

In [ ]:
print('shap_values: ' + str(len(shap_values)), shap_values)
print('shap_values2: '+ str(len(shap_values2)), shap_values2)
shap_values999 = explainer.shap_values(X.iloc[999, :], nsamples=500)
print('shap_values999: '+ str(len(shap_values999)), shap_values999)
shap_values999b = explainer.shap_values(X.iloc[999, :], nsamples=500)
print('shap_values999b: '+ str(len(shap_values999b)), shap_values999b)

In [ ]:
#shap.summary_plot(shap_values, X.iloc[0, :], plot_type="bar", feature_names=keyword)


In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
#shap.force_plot(explainer.expected_value[0], shap_values[0], X.iloc[299, :])
shap.force_plot(explainer.expected_value[0], shap_values[0], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[2], shap_values[2], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[3], shap_values[3], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[4], shap_values[4], keyword)

In [ ]:
#shap.force_plot(explainer.expected_value[5], shap_values[5], X.iloc[299, :])

In [ ]:
shap_values50 = explainer.shap_values(X.iloc[280:330, :], nsamples=500)

In [ ]:
shap.force_plot(explainer.expected_value[0], shap_values50[0], X.iloc[280:330, :],feature_names=keyword)

In [ ]:
shap.force_plot(explainer.expected_value[4], shap_values50[4], X.iloc[280:330, :],feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[0], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[2], X, plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[4], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)
print('shap_values50[4]: ', shap_values50[4].shape, shap_values50[4].sum(), shap_values50[4].min(), shap_values50[4].max())

In [ ]:
shap_values11 = explainer.shap_values(X.iloc[88888:88899, :], nsamples=500)

In [ ]:
shap.summary_plot(shap_values11[0], X.iloc[280:291, :], plot_type="bar", feature_names=keyword)

In [ ]:
print(shap_values, shap_values50, shap_values11)

In [ ]:
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print(shap_values_abs, shap_values_abs_sum, shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\n', feature_order)

In [ ]:
a = [[1, 2, 3, 4, 5, 6, -7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]]
nd_a = np.array(a)

#shap_values = nd_a
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print("\nshap_values: (shap_values)\n", shap_values, shap_values.dtype, shap_values.shape, "\nshap_values_abs: np.abs(shap_values)\n", shap_values_abs, 
      "\nshap_values_abs_sum: np.sum(shap_values_abs, axis=0)\n", shap_values_abs_sum, "\nshap_values_abs_sum_argsort: np.argsort(shap_values_abs_sum)\n", shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\nfeature_order:', feature_order)

In [ ]:
#shap.values[00,7)
shap00_list = [-0.00537375,  0.01205069,  0.04364897,  0.00142398, -0.00139138, -0.08626259,  0.01085853]
shap00_array = np.array(shap00_list)
print(shap00_array.dtype, shap00_array)

In [ ]:
B = np.array([[-9.,11.,-21.,63.,-252.],[3891.,506.,-1008.,3031.,-12117.],[3891.,576.,-1149.,3451.,-13801.],[3891.,-3891.,7782.,-23345.,93365.],[1024.,-1024.,2049.,-6144.,24572.]])
x = np.abs(B).argmax(axis=0)[0]
print(B, x)

In [ ]:
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print(shap_values, shap_values_abs, shap_values_abs_sum, shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\n', feature_order)

In [ ]:
shap.summary_plot(shap_values50[3], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50, X, plot_type="bar", feature_names=keyword)

In [ ]:
print(shap_values[1].sum(), X.shape)
#shap.summary_plot(shap_values, X, plot_type="bar", feature_names=keyword)

In [ ]:
shap_values50[0][0:5]

# ==== MAIN THREAD FROM 11/22: shap.KernelExplainer(model.predict, denseTest[0][0]) 

In [ ]:
train_generator.file_x_stack.shape

In [ ]:
X_train = train_generator.file_x_stack
y_train = test_generator.file_x_stack[0]
print(X_train)
print(X_train[0])
print(X_train[3].shape)
print(X_train.reshape(1, -1))
print(y_train)

In [ ]:
X_train_summary = shap.kmeans(train_generator.file_x_stack, 7)

In [ ]:
print(X_train_summary.data.shape)
print(X_train_summary.data)

In [ ]:
X_train_summary.data[6]

In [ ]:
model.summary()

In [ ]:
def convert_to_data(val, keep_index=False):
#    if isinstance(val, shap.common.Data):
#        return val
    if type(val) == np.ndarray:
        return DenseData(val, [str(i) for i in range(val.shape[1])])
    elif str(type(val)).endswith("'pandas.core.series.Series'>"):
        return DenseData(val.values.reshape((1,len(val))), list(val.index))
    elif str(type(val)).endswith("'pandas.core.frame.DataFrame'>"):
        if keep_index:
            return DenseDataWithIndex(val.values, list(val.columns), val.index.values, val.index.name)
        else:
            return DenseData(val.values, list(val.columns))
    elif sp.sparse.issparse(val):
        if not sp.sparse.isspmatrix_csr(val):
            val = val.tocsr()
        return SparseData(val)
    else:
        assert False, "Unknown type passed as data object: "+str(type(val))

In [ ]:
print(type(model))
print(model.name)
print(model.input)
print(model.output)
print(model.input.shape)
print(model.output.shape)

In [ ]:
print(model.weights)

In [ ]:
print(type(model))

In [ ]:
val = test_generator.file_x_stack
print(val.shape[1], range(val.shape[1]))
denseTest = shap.utils._legacy.DenseData(val, [str(i) for i in range(val.shape[1])])
print(denseTest.data)

In [ ]:
def my_predict(X):
    return model.predict([X[:, i] for i in range(X.shape[1])]).flatten()

In [ ]:
explainer = shap.KernelExplainer(my_predict, model.layers[0].input)
#explainer = shap.KernelExplainer(model.predict, model.layers[0].input)

#explainer = shap.KernelExplainer(model.predict, denseTest[0][0])
#explainer = shap.KernelExplainer(model.predict, test_generator.file_x_stack[0][0])

In [ ]:
print(predictions)
print(predictions.shape)
print(predictions.dtype)
print(predictions.min())
print(predictions.max())

In [ ]:
from sklearn.neural_network import MLPRegressor
from multi_path_fusion.src.data_generators.mpf_h_data_generator import MPF_H_DataGenerator
from tensorflow.keras.models import Sequential

#nn = MLPRegressor(solver="lbfgs", alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=0)
#nn.fit(X_train.reshape(1, -1), y_train)
#print_accuracy(nn.predict)

# explain all the predictions in the test set
#explainer = shap.KernelExplainer(model.predict, test_generator)
explainer = shap.KernelExplainer(model.predict, X_train_summary.data)
#test_generator
#shap_values = explainer.shap_values(X_test)
#shap.summary_plot(shap_values, X_test)

In [ ]:
from sklearn import linear_model

lin_regr = linear_model.LinearRegression()
lin_regr.fit(X_train, X_train_summary )

print_accuracy(lin_regr.predict)

# ==========================

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X, y = np.arange(10).reshape((5, 2)), range(5)
print(X)
print(list(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

In [ ]:
train_test_split(y, shuffle=False)

In [ ]:
train_test_split(y)

In [ ]:
x = np.arange(10)
x

In [ ]:
x.shape = (2, 5) 
x

In [ ]:
#print(x[0:])
print(x[0:,0:])

In [ ]:
mpf_bands = np.array([18, 14, 9, 3, 20, 8, 10])
mpf_bands

# ==========================

In [ ]:
print(train_generator.batch_size)
print(train_generator.branch_inputs)
print(len(train_generator.file_x_stack))
print(train_generator.file_x_stack.shape)
print(train_generator.file_x_stack[0].shape)
print(type(train_generator.file_x_stack))
print(train_generator.file_x_stack[0][799999])
print(train_generator.file_x_stack[3,0:])

In [ ]:
import pandas as pd

In [ ]:
data = train_generator.file_x_stack
df=pd.DataFrame(data=data[0:,0:], 
                index=[i for i in range(data.shape[0])],
                columns=['f'+str(i) for i in range(data.shape[1])])
df

In [ ]:
df.shape

In [ ]:
data = train_generator.file_x_stack
print(data.shape)
print(type(data))
df=pd.DataFrame(data=data[0:,0:], 
                index=[i for i in range(data.shape[0])],
                columns=['f'+str(i) for i in range(data.shape[1])])
df

In [ ]:
print(test_generator.batch_size)
print(test_generator.branch_inputs)
print(len(test_generator.file_x_stack))
print(test_generator.file_x_stack.shape)
print(test_generator.file_x_stack[0].shape)
print(type(test_generator.file_x_stack))
print(test_generator.file_x_stack[0][99999])

In [ ]:
data = test_generator.file_x_stack
df=pd.DataFrame(data=data[0:,0:], 
                index=[i for i in range(data.shape[0])],
                columns=['f'+str(i) for i in range(data.shape[1])])
df

In [ ]:
X = (train_generator.file_x_stack).astype(float)
X
print(type(X))

In [ ]:
print(train_generator.batch_size)
print(train_generator.branch_inputs)
print(len(train_generator.file_x_stack))
print(train_generator.file_x_stack.shape)
print(train_generator.file_x_stack[0].shape)
print(type(train_generator.file_x_stack))
print(train_generator.file_x_stack[0][799999])
print(train_generator.file_x_stack[4,0:])

In [ ]:
X_train_summary

In [ ]:
from sklearn.neural_network import MLPRegressor

nn = MLPRegressor(solver="lbfgs", alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=0)
nn.fit(X_train, y_train)
print_accuracy(nn.predict)

# explain all the predictions in the test set
explainer = shap.KernelExplainer(nn.predict, X_train_summary)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
explainer = shap.KernelExplainer(model.predict, X_train_summary)

In [ ]:
shap_values = explainer.shap_values(X_test,nsamples=100)

In [ ]:
def f(X):
    return model.predict([X[:, i] for i in range(X.shape[1])]).flatten()

In [ ]:
explainer = shap.KernelExplainer(f, test_generator.file_x_stack)

In [ ]:
explainer = shap.KernelExplainer(f, X.iloc[:50, :])
shap_values = explainer.shap_values(X.iloc[299, :], nsamples=500)
shap.force_plot(explainer.expected_value, shap_values, X_display.iloc[299, :])

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
import xgboost

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

In [ ]:
model.save('mpf_sequential_hd')

In [ ]:
reloaded_model = tf.keras.models.load_model('mpf_sequential_hd')

# * SHAP * it performs a perturbation around the points of the training dataset and calculates the impact of this perturbation to the model

## https://www.yourdatateacher.com/2021/05/17/how-to-explain-neural-networks-using-shap/

In [ ]:
#!pip install shap

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

In [ ]:
X,y = load_diabetes(return_X_y=True)
features = load_diabetes()['feature_names']

In [ ]:
features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test 

In [ ]:
model = make_pipeline(
    StandardScaler(),
    MLPRegressor(hidden_layer_sizes=(5,),activation='logistic', max_iter=10000,learning_rate='invscaling',random_state=0)
)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
explainer = shap.KernelExplainer(model.predict,X_train)

In [ ]:
print(len(features))
print(len(X_test))
print(X_test.mean())
print(X_test)

In [ ]:
shap_values = explainer.shap_values(X_test,nsamples=100)

In [ ]:
shap.summary_plot(shap_values,X_test,feature_names=features)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:]  ,X_test[0,:],feature_names=features)